# STAC (Simultaneous Tracking and Calibration) 

In this notebook, we will explore how STAC can register a biomechanical model of rats to 3D pose estimates obtained from DANNCE markerless pose estimation. This algorithm is designed to accurately track the movements of rats in three-dimensional space, which can be useful in a variety of research areas, including neuroscience, biophysics, and biomechanics.

STAC is an algorithm that can simultaneously calibrate a biomechanical model to keypoints by learning from data. We will demonstrate the different steps in STAC, including configuration files, model calibration, tracking, and visualization.

In [ ]:
%load_ext autoreload
%autoreload 2
from stac import compute_stac, viz
from IPython.display import Video
import numpy as np
from scipy.io import loadmat

In [ ]:
data_path = "data.mat"
param_path = "../params/params.yaml"
offset_path = "offset.p"
results_path = "results.p"
n_fitting_frames = 500
n_transform_frames = 1000

# Load data and convert from millimeters to meters
keypoints = loadmat(data_path)["pred"][:] / 1000

# Calibration

The calibration step is a crucial part of the STAC algorithm. It involves using a set of poses to calibrate the biomechanical model and determine the relationship between the model and the 3D pose estimates. This step ensures that the model accurately represents the movement of the rat. In this notebook, we will demonstrate how to `fit` the model using a short set of poses. While the original implementation of STAC called for joint optimization of tracking and calibration across the entire dataset, in practice we find that doing so on a small sample of data is sufficient. 

We recommend calibrating using approximately 500 contiguous samples (10 seconds at 50 Hz), which we will refer to as a trajectory. Calibrating this trajectory takes approximately 5 minutes on a Kaby Lake Xeon processor. If the calibration seems innacurate, it may help to select trajectories in which the animal is on all four limbs (if calibrating for rodents). 

In [ ]:
st = compute_stac.STAC(param_path)
st = st.fit(keypoints[:n_fitting_frames])
st.save(save_path=offset_path)

# Tracking

Now that we have calibrated the biomechanical model, we can use `transform` to track the rat's movements in 3D space. The algorithm optimizes a cost function that balances the error between the predicted and observed keypoints, while also considering the dynamics of the rat's movement. In this notebook, we will demonstrate how to set up the tracking configuration, load the data, and run the optimization. We will also discuss how to fine-tune the tracking parameters to achieve the best possible results.

In [ ]:
data = st.transform(keypoints[:n_transform_frames], offset_path)
st.save(save_path=results_path)

# Results

You can render videos using stac's visualization tools. There are two major rendering options, `render_mujoco` and `render_overlay`. 

The `render_mujoco` option renders a video of the stac reconstruction as it appears in a user-specified camera in MuJoCo. 

The `render_overlay` option grabs a user-specified rgb video and overlays a rendering of the skeletal model in mujoco from the perspective of the camera from which the video was taken. This is convenient to compare the skeletal registration with the pose of the real animal. 

In [ ]:
save_path = "mujoco_video.mp4"
viz.render_mujoco(
    param_path,
    results_path,
    save_path,
    frames = np.arange(n_transform_frames),
)
Video(save_path)

You can use `render_overlay` to render a video of the stac reconstruction on top of your original images.  

In [ ]:
save_path = "overlay_video.mp4"
calibration_path = 'calibration.mat'
video_path = './videos/Camera2/0.mp4'
viz.render_overlay(
    param_path,
    video_path,
    results_path,
    save_path,
    frames=np.arange(n_transform_frames),
    camera="Camera2",
    calibration_path=calibration_path,
    segmented=True
    )
Video(save_path)